In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

import datetime
import time

#import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
# To scrap Residential Indoor Lighting > New Indoor Products > All Products

def residential_indoor_lighting():
    
    category = ''
    sub_categories = []
    p_data = []
    p_name, prices, desc = [], [], []
    
    # Use base URL for Lighting & Controls
    url = "https://www.hubbell.com/Products/Lighting-Controls/cl/535409"
    driver = webdriver.Chrome()
    dcap = dict(DesiredCapabilities.CHROME)
    dcap["chrome.page.settings.userAgent"] = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362")
    driver = webdriver.Chrome(desired_capabilities = dcap, service_args = ['--ignore-ssl-errors=true'])
    driver.implicitly_wait(20)
    driver.get(url)
    
    # Get Residential Indoor Lighting page
    link = driver.find_element(By.CLASS_NAME, value='category-card')
    category = link.text
    print(f"Clicking: {link.text}")
    link.click()
    time.sleep(5)
    
    # Getting list of all sub-category under Residential Indoor Lighting
    links = driver.find_elements_by_xpath('//*[@class="category-card"]/a')
    nm = driver.find_elements_by_xpath('//*[@class="category-card"]/a/div[2]')
    
    # Iterating over sub-categories
    for i, s_c in zip(range(0,5), links[0:5]):
        p_nm, pr, de = [], [], []
        #print(driver.window_handles)
        sub_categories.append(nm[i].text)
        o_page = driver.window_handles[0]
        print(f"Clicking: {s_c.text}")
        ActionChains(driver).key_down(Keys.CONTROL).click(s_c).key_up(Keys.CONTROL).perform()
        time.sleep(5)
        
        print(driver.window_handles)
        
        # Checking if new tab is opened
        if len(driver.window_handles) > 1:
            n_page = driver.window_handles[-1]
            driver.switch_to.window(n_page)
            
            # Checking if specified element exists
            if driver.find_elements(By.CLASS_NAME, value="utilities-results"):
                n_p = driver.find_element(By.CLASS_NAME, value="utilities-results")
                n_p = int(n_p.text.split(' ')[0])
                n_pages = n_p // 30
            else:
                sub_categories.pop()
                driver.close()
                driver.switch_to.window(o_page)
                continue
            
            # Iterating n_pages times to get details from multiple pages
            for _ in range(1):
                products = driver.find_elements_by_xpath('//*[@class="product-name"]')
                cost = driver.find_elements_by_xpath('//*[@class="product-action-container"]/div[1]/span[2]')
                description = driver.find_elements_by_xpath('//*[@class="product-description"]')
                
                # Iterating over all products in a current page (Maximum 30)
                for n, p in enumerate(zip(products[:2], cost[:2], description[:2])):
                    o_p = driver.window_handles[1]
                    print(f"Clicking Details of product # {n}")
                    ActionChains(driver).key_down(Keys.CONTROL).click(p[0]).key_up(Keys.CONTROL).perform()
                    time.sleep(5)
                    
                    if len(driver.window_handles) > 2:
                        p_nm.append(p[0].text)
                        pr.append(p[1].text)
                        de.append(p[2].text)
                        
                        print(driver.window_handles)
                        n_p = driver.window_handles[-1]
                        driver.switch_to.window(n_p)
                        
                        tables = pd.read_html(driver.page_source)
                        p_data.append(tables)
                        
                        print(f"Inside Details page of product # {n}")
                        time.sleep(5)
                        
                        driver.close()
                        driver.switch_to.window(o_p)
            
            driver.close()
            driver.switch_to.window(o_page)
        
        p_name.append(p_nm)
        prices.append(pr)
        desc.append(de)
        
    driver.quit()
        
    return (category, sub_categories, p_name, prices, desc, p_data)

In [3]:
data = residential_indoor_lighting()

Clicking: Residential Indoor Lighting
Clicking: New Indoor Products
['EB39B4E402C0B514C6136F04D64D0CEA', '024B50C169AD4881D8C6E91629A7EE34']
Clicking Details of product # 0
['EB39B4E402C0B514C6136F04D64D0CEA', '024B50C169AD4881D8C6E91629A7EE34', 'EDC2A723D1E611287015264FC56ADBF3']
Inside Details page of product # 0
Clicking Details of product # 1
Clicking: LED
['EB39B4E402C0B514C6136F04D64D0CEA', '6815CFBA0221F9C7B342969136CA8B16']
Clicking Details of product # 0
['EB39B4E402C0B514C6136F04D64D0CEA', '6815CFBA0221F9C7B342969136CA8B16', '86EED8E8CC55E508CBE080A616ED749C']
Inside Details page of product # 0
Clicking Details of product # 1
Clicking: Chandeliers
['EB39B4E402C0B514C6136F04D64D0CEA', '534DF24FF0622F579EF5CC534A67AE53']
Clicking Details of product # 0
['EB39B4E402C0B514C6136F04D64D0CEA', '534DF24FF0622F579EF5CC534A67AE53', 'EB2CF10BB0ACBCC66C1C66D4F01F7907']
Inside Details page of product # 0
Clicking Details of product # 1
Clicking: Pendants
['EB39B4E402C0B514C6136F04D64D0CEA

In [104]:
# Preparing the final dataframe
category = data[0]
sub_cat = data[1][:len(data[5])]
p_name = [j for i in data[2] for j in i]
desc = [j for i in data[4] for j in i]
price = [j for i in data[3] for j in i]
p_data = [pd.concat(i, sort=False, ignore_index=True) for i in data[5]]


df = pd.DataFrame({
    'Category': [category]*sum(list(map(lambda x: len(x), p_data))),
    'Sub-Category': np.repeat(sub_cat, list(map(lambda x: len(x), p_data))),
    'Name': np.repeat(p_name, list(map(lambda x: len(x), p_data))),
    'Description': np.repeat(desc, list(map(lambda x: len(x), p_data))),
    'Price': np.repeat(price, list(map(lambda x: len(x), p_data))),
    'Metric': pd.concat(p_data, sort=False, ignore_index=True)[0],
    'Value': pd.concat(p_data, sort=False, ignore_index=True)[1]
})

In [105]:
print(df.shape)
df.head()

(193, 7)


,Category,Sub-Category,Name,Description,Price,Metric,Value
0,Residential Indoor Lighting,New Indoor Products,"7"" Integrated LED Surface Mount Light, 24-Pack",Homestyle Surface Mount LED,$433.40 USD,Can be mounted up or down,No
1,Residential Indoor Lighting,New Indoor Products,"7"" Integrated LED Surface Mount Light, 24-Pack",Homestyle Surface Mount LED,$433.40 USD,Color,White
2,Residential Indoor Lighting,New Indoor Products,"7"" Integrated LED Surface Mount Light, 24-Pack",Homestyle Surface Mount LED,$433.40 USD,Color Generic,White
3,Residential Indoor Lighting,New Indoor Products,"7"" Integrated LED Surface Mount Light, 24-Pack",Homestyle Surface Mount LED,$433.40 USD,Color Rendering Index (CRI),90 CRI
4,Residential Indoor Lighting,New Indoor Products,"7"" Integrated LED Surface Mount Light, 24-Pack",Homestyle Surface Mount LED,$433.40 USD,Color Temperature,3000 K
